In [ ]:
%pip install python-dotenv
%pip install pymongo
%pip install web3

Información relacionada:
- Web3 API: https://web3py.readthedocs.io/en/stable/web3.main.html

Importación de librerias

In [43]:
import pymongo
import os

from dotenv import load_dotenv
from decimal import Decimal

In [37]:
# Carga los secretos del archivo .env
load_dotenv()

# Accede a los valores definidos en .env
URL_INFURA = os.getenv("URL_INFURA")
MONGODB_SERVER = os.getenv("MONGODB_SERVER")

print(URL_INFURA[:27]+'*******'+URL_INFURA[-4:])
print(MONGODB_SERVER)


https://mainnet.infura.io/v*******27a1
mongodb://localhost:27017/


Conexión API Infura

In [29]:
from web3 import Web3

try:
    w3 = Web3(Web3.HTTPProvider(URL_INFURA))
except Exception as e:
    print("An error occurred:", str(e))

try:
    client_ver = w3.client_version
    print("Conectado a la red de Ethereum. Cliente: ", client_ver)
except Exception as e:
    print("ERROR. No ha sido posible establecer la conección: ", e)

Conectado a la red de Ethereum. Cliente:  Geth/v1.13.12-omnibus-e0f86448/linux-amd64/go1.21.6


Función para la obtención de saldos

In [31]:
# block_number (OPCIONAL): Número de bloque para el que se desea consultar el saldo (Si no se especifica, devuelve el saldo actual)
def get_balance(address, block_number='latest'):
    balance = w3.eth.get_balance(address, block_identifier=block_number)
    return w3.from_wei(balance, 'ether')

# Prueba
address = '0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a' # Reemplaza con la dirección que deseas consultar
block_number = 'latest' # Reemplaza con el número de bloque que deseas consultar
print('Test > Adress: ', address, '| Balance: ', get_balance(address, block_number))

Test > Adress:  0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a | Balance:  1498474.270654427190437905


### Conexión a Base de Datos en MongoDB

In [38]:
mongoClient = pymongo.MongoClient(MONGODB_SERVER, serverSelectionTimeoutMS=5000)

try:
    print(mongoClient.server_info())
except Exception:
    print("Unable to connect to the server.")

{'version': '5.0.3', 'gitVersion': '657fea5a61a74d7a79df7aff8e4bcf0bc742b748', 'targetMinOS': 'Windows 7/Windows Server 2008 R2', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 3, 0], 'openssl': {'running': 'Windows SChannel'}, 'buildEnvironment': {'distmod': 'windows', 'distarch': 'x86_64', 'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.26.28806 for x64', 'ccflags': '/nologo /WX /FImongo/platform/basic.h /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++17 /Gw /Gy /Zc:inline', 'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.26.28806 for x64', 'cxxflags': '/TP', 'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF', 'target_arch': 'x86_64', 'target_os': 'windows', 'cppdefine

Selección colecciones de MongoDB y definición de indices

In [52]:
mongoDb = mongoClient['web3_tfm']
collection_block_history = mongoDb['block_history']
collection_account_directory = mongoDb['account_directory']

# Colección
collection_balances = mongoDb['account_balances']

# Indices
collection_balances.create_index('address')

# Define un índice único sobre los campos 'address', 'network' y 'detail.detail_date'
collection_balances.create_index([('address', 1), ('network', 1), ('detail.detail_date', 1)], unique=True)

'address_1_network_1_detail.detail_date_1'

Selecciona unicamente las bloques a partir del año 2023

In [41]:
# Realizar la consulta para obtener documentos con 'date' > '2022-12-31'
documentos = collection_block_history.find({'date': {'$gt': '2022-12-31'}})

In [42]:
blocks = {doc['max_block_number']:doc['date'] for doc in documentos}
print("Número de elementos recuperados: ", len(blocks))

Número de elementos recuperados:  411


Función que obtiene los saldos de una dirección para todos los bloques de la lista 

In [67]:
def insert_balances_address(address, blocks, from_network, to_network):
    
    balance = 0
    print(blocks)
    balances = []
    print('>> Address: ', address)

    # Obtiene el saldo para cada uno de los bloques fin dia (último bloque en una fecha)
    for block, date in blocks.items():
        
        balance = get_balance(address, block)
        print('block:', block, date, '| balance:', balance)
        
        balances.append({
            'date': date,
            'block_number': block,
            'balance': balance
        })
        
    # Prepara estructura documento a insertar
    doc = {
        'from_network': from_network,
        'to_network': to_network,
        'address': address,
        'block_number': 0,
        'date': '',
        'balance': 0,
        'detail': []
    }

    # Incluye la información de los saldos en el documento
    # En doc['detail'] incluirá el historico de saldos
    for elem in balances:
        
        # Convertir a punto flotante
        balance_float = float(str(elem['balance']))
        
        doc['date'] = elem['date']
        doc['balance'] = balance_float
        doc['block_number'] = elem['block_number']
        doc['detail'].append({
            'detail_date': elem['date'],
            'detail_balance': balance_float
        })
    
    # Prepara filtro para validar si ya existe información para la misma cuenta y misma red blockchain
    filtro = {'from_network': doc['from_network'], 'address': doc['address']}

    # Actualiza documento (en caso de no existir, inserta el documento)
    # La opción upsert=True indica que se debe insertar un nuevo documento si no se encuentra ningún documento 
    # que coincida con el filtro
    collection_balances.update_one(filtro, {'$set': doc}, upsert=True)
    
    return balances

Busca las cuentas para las que se debe almacenar el sado (isBalance = True)

In [76]:
# Consulta para recuperar las cuentas con "isBalance" igual a True
query = {"isBalance": True}
result = collection_account_directory.find(query)

accounts_balance = [
    {
        'address': cuenta['address'], 
        'from_network': cuenta['from_network'],
        'to_network': cuenta['to_network'],
        'full_description': cuenta['full_description']
    } 
    for cuenta in result
]

print(accounts_balance)

[{'address': '0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a', 'from_network': 1, 'to_network': 42170, 'full_description': 'Arbitrum: Bridge'}, {'address': '0xbEb5Fc579115071764c7423A4f12eDde41f106Ed', 'from_network': 1, 'to_network': 10, 'full_description': 'Optimism - OptimismPortal'}, {'address': '0x49048044D57e1C92A77f79988d21Fa8fAF74E97e', 'from_network': 1, 'to_network': 8453, 'full_description': 'Base - OptimismPortal'}, {'address': '0xd19d4B5d358258f05D7B411E21A1460D11B0876F', 'from_network': 1, 'to_network': 59144, 'full_description': 'Linea - L1 Message Service'}]


### Carga los saldos de todas las cuentas con la marca "isBalance"

In [ ]:
for acc_details in accounts_balance:
    
    print('Account: ', acc_details['full_description'])
    
    insert_balances_address(  
        acc_details['address'], 
        blocks, 
        acc_details['from_network'], 
        acc_details['to_network']
    )